## Lec3. Higher-Order Functions

### 1.Functions as return values

In [3]:
def make_adder(n):
  def adder(x):
      return x + n
  return adder

f = make_adder(3)
print(f(4))
make_adder(3)(4) # make_adder can work as mul

7


7

### 2.Higher-Order Functions

KEY: takes functions and returns functions

In [10]:
# Example: Sound

from wave import open
from struct import Struct
from math import floor

frame_rate = 11025

def encode(x):
    """Encode float x between -1 and 1 as two bytes.
    (See https://docs.python.org/3/library/struct.html)
    """
    i = int(16384 * x)
    return Struct('h').pack(i)

def play(sampler, name='song.wav', seconds=2):
    """Write the output of a sampler function as a wav file.
    (See https://docs.python.org/3/library/wave.html)
    """
    out = open(name, 'wb')
    out.setnchannels(1)
    out.setsampwidth(2)
    out.setframerate(frame_rate)
    t = 0
    while t < seconds * frame_rate:
        sample = sampler(t)
        out.writeframes(encode(sample))
        t = t + 1
    out.close()

def tri(frequency, amplitude=0.3):
    """A continuous triangle wave."""
    period = frame_rate // frequency
    def sampler(t):
        saw_wave = t / period - floor(t / period + 0.5)
        tri_wave = 2 * abs(2 * saw_wave) - 1
        return amplitude * tri_wave
    return sampler

c_freq, e_freq, g_freq = 261.63, 329.63, 392.00

play(tri(e_freq))

def note(f, start, end, fade=.01):
    """Play f for a fixed duration."""
    def sampler(t):
        seconds = t / frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        elif seconds < start + fade:
            return (seconds - start) / fade * f(t)
        elif seconds > end - fade:
            return (end - seconds) / fade * f(t)
        else:
            return f(t)
    return sampler

play(note(tri(e_freq), 1, 1.5))

def both(f, g):
    return lambda t: f(t) + g(t)

c = tri(c_freq)
e = tri(e_freq)
g = tri(g_freq)
low_g = tri(g_freq / 2)

play(both(note(e, 0, 1/8), note(low_g, 1/8, 3/8)))

play(both(note(c, 0, 1), both(note(e, 0, 1), note(g, 0, 1))))

def mario(c, e, g, low_g):
    z = 0
    song = note(e, z, z + 1/8)
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(c, z, z + 1/8))
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(g, z, z + 1/4))
    z += 1/2
    song = both(song, note(low_g, z, z + 1/4))
    return song

def mario_at(octave):
    c = tri(octave * c_freq)
    e = tri(octave * e_freq)
    g = tri(octave * g_freq)
    low_g = tri(octave * g_freq / 2)
    return mario(c, e, g, low_g)

play(mario_at(1))


In [11]:
# function composition
def make_adder(n):
  def adder(x):
      return x + n
  return adder

def square(x):
  return x * x

def triple(x):
  return 3 * x

def composel(f, g):
  def h(x):
    return f(g(x))
  return h


### 3.lambda expressions

In [26]:
# example
square = lambda x: x * x
'''a function with formal parameter x that returns the value of "x * x" '''
print(square(3)) # equals to:
print((lambda x: x * x)(3))


9
9


### 4.function currying

A way of manipulating functions.

In [14]:
# make_adder as an example
def make_adder(n):
  return lambda k : n + k
from operator import add
print(make_adder(2)(3))
print(add(2,3))

5
5


There is a general description of the relationship between the two functions above.

In [9]:
'''
def curry2(f):
  def g(x):
    def h(y):
      return f(x, y)
    return h
  return g
''' 
from operator import add
curry2 = lambda f: lambda x: lambda y: f(x, y)
fun = curry2(add)
print(fun(3)(2))
add_three = fun(3)
print(add_three(2))

5
5


### 5.Funuction Decorators

In [18]:

def trace(f):
  def traced(x):
    print("calling",f,"on argument", x)
    return f(x)
  return traced

@trace
def square (x):
  return x*x
# equal to: square = trace(square)
@trace
def sum_squares_up_to(n):
  k = 1
  total = 0
  while k <= n:
    total, k = total + square(k), k + 1
  return total
 
sum_squares_up_to(4)



calling <function sum_squares_up_to at 0x00000181A2E6B1F0> on argument 4
calling <function square at 0x00000181A2E17E50> on argument 1
calling <function square at 0x00000181A2E17E50> on argument 2
calling <function square at 0x00000181A2E17E50> on argument 3
calling <function square at 0x00000181A2E17E50> on argument 4


30

A decorator serves as a shortcut to transform one function into another, based on the higher-order function referred in the `@` statement.